In [51]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np

In [52]:
train_dir = "../train"
test_dir = "../test"

print(train_dir)
print(test_dir)


../train
../test


In [53]:
# Carregue os datasets
img_size = (64, 64)  # Tamanho desejado (ex: 224x224 para Transfer Learning)
batch_size = 32        # Pode ajustar conforme sua GPU

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,       # Embaralha os dados
    seed=42             # Para reprodutibilidade
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False       # Não precisa embaralhar o teste
)

Found 34714 files belonging to 21 classes.
Found 11548 files belonging to 21 classes.


In [54]:
# Número de classes (letras)
class_names = train_ds.class_names
print("Classes (letras) encontradas:", class_names)

# Formato de um batch de imagens
for images, labels in train_ds.take(1):  # Pega o primeiro batch
    print("\nFormato das imagens:", images.shape)  # (batch_size, 224, 224, 3)
    print("Formato de UMA imagem:", images[0].shape)  # (224, 224, 3)
    print("Rótulos do batch:", labels.numpy())  # Ex: [0, 2, 1, ...] (índices das classes)

Classes (letras) encontradas: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y']

Formato das imagens: (32, 64, 64, 3)
Formato de UMA imagem: (64, 64, 3)
Rótulos do batch: [ 7 13  6 19 18 13  1  6  3 15  1 18  9  4  1 11  9 17  5  0 12  4 10 19
 13  4 17  1  3 18  1  5]


In [55]:
print("GPUs disponíveis:", tf.config.list_physical_devices('GPU'))

GPUs disponíveis: []


In [56]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [57]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=10)

Epoch 1/10
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 33s 30ms/step - accuracy: 0.5837 - loss: 1.7169 - val_accuracy: 0.7950 - val_loss: 0.7685
Epoch 2/10
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 33s 30ms/step - accuracy: 0.9375 - loss: 0.2603 - val_accuracy: 0.8328 - val_loss: 0.6303
Epoch 3/10
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 33s 30ms/step - accuracy: 0.9775 - loss: 0.1016 - val_accuracy: 0.8369 - val_loss: 0.6390
Epoch 4/10
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 33s 30ms/step - accuracy: 0.9904 - loss: 0.0464 - val_accuracy: 0.8477 - val_loss: 0.6313
Epoch 5/10
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 33s 30ms/step - accuracy: 0.9963 - loss: 0.0219 - val_accuracy: 0.8406 - val_loss: 0.7116
Epoch 6/10
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 33s 30ms/step - accuracy: 0.9983 - loss: 0.0114 - val_accuracy: 0.8479 - val_loss: 0.7082
Epoch 7/10
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 33s 31ms/step - accuracy: 0.9989 - loss: 0.0075 - val_accuracy: 0.8518 - val_loss: 0.7626
Epoch 8/10
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 32s 29ms/step - accuracy: 0.9994 -

In [58]:
# Avaliação no conjunto de teste
test_loss, test_accuracy = model.evaluate(test_ds)

print(f"Acurácia no teste: {test_accuracy:.2%}")
print(f"Perda no teste: {test_loss:.4f}")

361/361 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8565 - loss: 0.8160
Acurácia no teste: 84.87%
Perda no teste: 0.8618


In [59]:
model.save("modelo_libras.h5")